In [12]:
import pandas as pd
import numpy as np
import os
import glob
import sys
import nilearn.signal as nls
import matplotlib.pyplot as plt
from scipy.stats import zscore as zscore

In [2]:
path='/media/data/HCPAging/data/3T_REST/'

In [5]:
files = sorted(glob.glob(path+'timecourse_tables'+'/'+'*'+'_rest_timecourse.csv'))
print(files[0])

/media/data/HCPAging/data/3T_REST/timecourse_tables/HCA6002236_V1_MR_rest_timecourse.csv


In [10]:
timecourses = {}
for file in files:
    timecourses[file.split('/')[-1].split('_')[0]] = pd.read_csv(file, index_col=0)

In [20]:
timecourses_filt = {}
for key in timecourses.keys():
    timecourses_filt[key] = pd.DataFrame(nls.clean(timecourses[key].T.values, t_r=0.8, detrend=False, high_pass=0.008),
                                  columns=timecourses[key].T.columns)

In [23]:
timecourses_filt_corr_z = {}
dct_group_task_FC = {}

for key in timecourses.keys():
    
    ### Calculate correlation matrix and make r-to-z transformation
    timecourses_filt_corr_z[key] = np.arctanh(timecourses_filt[key].corr()) # r-to-z
    
    ### Transform matrix to string for the subject
    dfg = timecourses_filt_corr_z[key].where(np.triu(np.ones(timecourses_filt_corr_z[key].shape), k=1).astype(bool)).stack().reset_index()
    indx = [i+'_&_'+j for i,j in zip(dfg['level_0'], dfg['level_1'])]
    fc_string = pd.Series(np.array(dfg[0]),index=indx)
    
    ### write result into group table
    dct_group_task_FC[key] = fc_string

df_group_task_FC = pd.DataFrame(dct_group_task_FC)

In [27]:
df_group_task_FC.T.to_csv(path+'FC_rest_group_z_hpass.csv')

In [ ]:
# read  existing sibject id
sID = pd.read_csv('/media/data/HCPAging/data/MLTablesMultCon/demography.csv', index_col=0).index.values
#print(len(sID))

In [31]:
df_group_task_FC.T.reindex(index=sID).to_csv('/media/data/HCPAging/data/MLTablesMultCon/rest_hpass.csv')